# UNet with Idealized Grass Dataset
### [Link to MLFlow Dashboard](http://0.0.0.0:5001)

## Imports

In [1]:
from __future__ import unicode_literals, print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from torch.utils import data
import itertools
import re
import random
import time
from unet.unets import U_net
from torch.autograd import Variable
# from penalty import DivergenceLoss
from unet.utils_unet import train_epoch, eval_epoch, test_epoch
from unet.dataset import IdealizedGrasslands
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
import warnings
warnings.filterwarnings("ignore")
import mlflow

cpu


## Training

In [ ]:
mlflow.end_run()
mlflow.set_experiment("UNet")
mlflow.start_run()

train_direc="./train/"
test_direc="./test/"
min_mse=10
output_length=100
input_length=7
learning_rate=0.001
dropout_rate=0
kernel_size=3
batch_size=1
max_epochs=1

train_indices=list(range(0,3))
valid_indices = list(range(3, 5))
# test_indices = list(range(600, 650))
model=U_net(input_channels = input_length, output_channels = 1, kernel_size = kernel_size,
            dropout_rate = dropout_rate).to(device)
train_set = IdealizedGrasslands(train_indices, input_length , 15, output_length, train_direc, file='uniform-pgml-success_list_simulation_runs.csv')
valid_set =IdealizedGrasslands(valid_indices, input_length , 15, output_length, test_direc, file='uniform-pgml-success_list_simulation_runs.csv')
train_loader = data.DataLoader(train_set, batch_size = batch_size, shuffle = True, num_workers = 1)
valid_loader = data.DataLoader(valid_set, batch_size = batch_size, shuffle = False, num_workers = 1)
loss_fun = torch.nn.L1Loss()
#regularizer = DivergenceLoss(torch.nn.MSELoss())

optimizer = torch.optim.Adam(model.parameters(), learning_rate, betas = (0.9, 0.999), weight_decay = 4e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 1, gamma = 0.9)

# Log parameters into MLFlow
mlflow.log_param("learning_rate", learning_rate)
mlflow.log_param("batch_size", batch_size)
mlflow.log_param("loss_function", loss_fun)
mlflow.log_param("max_epochs", max_epochs)
mlflow.log_param("optimizer", str(optimizer))
mlflow.log_param("scheduler", str(scheduler))


train_mse = []
valid_mse = []
test_mse = []
for i in range(max_epochs):
    mlflow.log_metric("Current Training Epoch", i + 1)
    print(f'Epoch {i} started')
    start = time.time()
    torch.cuda.empty_cache()
    scheduler.step()
    model.train()
    teacher_force_ratio=np.maximum(0, 1 - i * 0.03)
    train_loss = train_epoch(train_loader, model, optimizer, loss_fun, teacher_force_ratio)
    train_mse.append(train_loss)
    model.eval()
    mse, preds, trues = eval_epoch(valid_loader, model, loss_fun)
    valid_mse.append(mse)

    # send training metrics to MLFlow
    mlflow.log_metric("Epoch Loss", train_loss, step=i)
    mlflow.log_metric("Epoch Validation", mse, step=i)
    
    if valid_mse[-1] < min_mse:
        min_mse = valid_mse[-1]
        best_model = model
        torch.save(best_model, "unet_model2.pth")
    end = time.time()
    if (len(train_mse) > 50 and np.mean(valid_mse[-5:]) >= np.mean(valid_mse[-10:-5])):
            break
    print(train_mse[-1], valid_mse[-1], round((end-start)/60,5))
    print(f'Epoch {i} ended')

mlflow.log_artifact("unet_model2.pth")
mlflow.end_run()